Найти данные для задачи классификации или для задачи регрессии.

In [11]:
!pip install catboost
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from sklearn import tree

from sklearn.datasets import load_breast_cancer

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
import catboost as cb

warnings.filterwarnings('ignore')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
data = load_breast_cancer()

predicts = data.data
target = data.target

print(predicts)

print(target)

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 

In [13]:
A_train, A_test, y_train, y_test = train_test_split(predicts, target, test_size=0.8)
print(A_train.shape)
print(A_test.shape)

(113, 30)
(456, 30)


Реализовать баггинг

In [14]:
random_forest = RandomForestClassifier(max_depth=15, min_samples_split=10).fit(A_train, y_train)

y_preds_d = random_forest.predict(A_train)
print('F1 мера для тренировочных данных ', f1_score(y_preds_d, y_train, average='macro'))

y_pred = random_forest.predict(A_test)
print('F1 мера для тестовых данных ', f1_score(y_pred, y_test, average='macro'))


F1 мера для тренировочных данных  1.0
F1 мера для тестовых данных  0.944871666833117


In [15]:
random_forest = RandomForestClassifier()

params_grid = {
    'max_depth': [12, 18],
    'min_samples_leaf': [3, 10],
    'min_samples_split': [6, 12]
}

grid_search_random_forest = GridSearchCV(estimator=random_forest,
                                         param_grid=params_grid,
                                         scoring='f1_macro',
                                         cv=4)

grid_search_random_forest.fit(A_train, y_train)

best_model = grid_search_random_forest.best_estimator_

y_preds_d = best_model.predict(A_train)
print('F1 мера для тренировочных данных ', f1_score(y_preds_d, y_train, average='macro'))

y_pred = best_model.predict(A_test)
print('F1 мера для тестовых данных ', f1_score(y_pred, y_test, average='macro'))


F1 мера для тренировочных данных  0.9908017908017908
F1 мера для тестовых данных  0.9209028263257766


Реализовать бустинг на тех же данных, что использовались для баггинга.

In [16]:
model_catboost_clf = cb.CatBoostClassifier(iterations = 3000,
                                           task_type = 'GPU',
                                           devices = '0')

model_catboost_clf.fit(A_train, y_train)

Learning rate set to 0.014202
0:	learn: 0.6663498	total: 33.2ms	remaining: 1m 39s
1:	learn: 0.6423921	total: 84.5ms	remaining: 2m 6s
2:	learn: 0.6172707	total: 107ms	remaining: 1m 46s
3:	learn: 0.6031726	total: 154ms	remaining: 1m 55s
4:	learn: 0.5817364	total: 205ms	remaining: 2m 2s
5:	learn: 0.5590606	total: 226ms	remaining: 1m 52s
6:	learn: 0.5436990	total: 241ms	remaining: 1m 43s
7:	learn: 0.5258288	total: 271ms	remaining: 1m 41s
8:	learn: 0.5082067	total: 316ms	remaining: 1m 44s
9:	learn: 0.4919026	total: 351ms	remaining: 1m 45s
10:	learn: 0.4807841	total: 382ms	remaining: 1m 43s
11:	learn: 0.4614208	total: 416ms	remaining: 1m 43s
12:	learn: 0.4443112	total: 441ms	remaining: 1m 41s
13:	learn: 0.4300725	total: 466ms	remaining: 1m 39s
14:	learn: 0.4140773	total: 485ms	remaining: 1m 36s
15:	learn: 0.4008194	total: 499ms	remaining: 1m 33s
16:	learn: 0.3869944	total: 538ms	remaining: 1m 34s
17:	learn: 0.3785816	total: 563ms	remaining: 1m 33s
18:	learn: 0.3683281	total: 588ms	remaining:

In [18]:
y_preds_t = model_catboost_clf.predict(A_train, task_type='CPU')
print('F1 мера для тренировочных данных ', f1_score(y_preds_t, y_train, average='macro'))

y_preds = model_catboost_clf.predict(A_test, task_type='CPU')
print('F1 мера для тестовых данных ', f1_score(y_preds, y_test, average='macro'))

F1 мера для тренировочных данных  1.0
F1 мера для тестовых данных  0.9544592030360531
